In [ ]:
import cv2
import numpy as np

# -----------------------------
#  Load binary image
# -----------------------------
img = cv2.imread("img1.tif", cv2.IMREAD_GRAYSCALE)

# Threshold to ensure binary (0 / 255)
_, img_bin = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

# -----------------------------
#  Create structuring element
# -----------------------------
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

# -----------------------------
#  Basic morphological operations
# -----------------------------
img_erosion = cv2.erode(img_bin, kernel, iterations=1)
img_dilation = cv2.dilate(img_bin, kernel, iterations=1)

img_opening = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, kernel)
img_closing = cv2.morphologyEx(img_bin, cv2.MORPH_CLOSE, kernel)

# -----------------------------
#  Hit-or-Miss Transform
#  (requires strictly binary 0/1 image)
# -----------------------------
img_bin_hmt = img_bin // 255  # convert to 0/1

# Example structuring elements
J = np.array([[0, 1, 0],
              [1, 1, 1],
              [0, 1, 0]], dtype=np.uint8)

K = np.array([[1, 0, 1],
              [0, 0, 0],
              [1, 0, 1]], dtype=np.uint8)

hitormiss = cv2.morphologyEx(img_bin_hmt, cv2.MORPH_HITMISS, J)

# rescale to 0–255 for display
hitormiss = (hitormiss * 255).astype(np.uint8)

# -----------------------------
#  Boundary Extraction
#  Boundary = Image – Erosion(Image)
# -----------------------------
boundary = cv2.subtract(img_bin, img_erosion)

# -----------------------------
#  Show results
# -----------------------------
cv2.imshow("Original", img_bin)
cv2.imshow("Erosion", img_erosion)
cv2.imshow("Dilation", img_dilation)
cv2.imshow("Opening", img_opening)
cv2.imshow("Closing", img_closing)
cv2.imshow("Hit-or-Miss", hitormiss)
cv2.imshow("Boundary", boundary)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# -----------------------------
#  Load binary image
# -----------------------------
img = cv2.imread("img1.tif", cv2.IMREAD_GRAYSCALE)

# Threshold to ensure binary (0 / 255)
_, img_bin = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

# -----------------------------
#  Create structuring element
# -----------------------------
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

# -----------------------------
#  Basic morphological operations
# -----------------------------
img_erosion = cv2.erode(img_bin, kernel, iterations=1)
img_dilation = cv2.dilate(img_bin, kernel, iterations=1)

img_opening = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, kernel)
img_closing = cv2.morphologyEx(img_bin, cv2.MORPH_CLOSE, kernel)

# -----------------------------
#  Hit-or-Miss Transform
#  (requires strictly binary 0/1 image)
# -----------------------------
img_bin_hmt = img_bin // 255  # convert to 0/1

# Example structuring elements
J = np.array([[0, 1, 0],
              [1, 1, 1],
              [0, 1, 0]], dtype=np.uint8)

K = np.array([[1, 0, 1],
              [0, 0, 0],
              [1, 0, 1]], dtype=np.uint8)

hitormiss = cv2.morphologyEx(img_bin_hmt, cv2.MORPH_HITMISS, J)

# rescale to 0–255 for display
hitormiss = (hitormiss * 255).astype(np.uint8)

# -----------------------------
#  Boundary Extraction
#  Boundary = Image – Erosion(Image)
# -----------------------------
boundary = cv2.subtract(img_bin, img_erosion)

# -----------------------------
#  Show results
# -----------------------------
cv2.imshow("Original", img_bin)
cv2.imshow("Erosion", img_erosion)
cv2.imshow("Dilation", img_dilation)
cv2.imshow("Opening", img_opening)
cv2.imshow("Closing", img_closing)
cv2.imshow("Hit-or-Miss", hitormiss)
cv2.imshow("Boundary", boundary)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# ------------------------------------
# Load grayscale image
# ------------------------------------
img = cv2.imread("img2.tif", cv2.IMREAD_GRAYSCALE)

# ------------------------------------
# Threshold to isolate bright melt pool
# (Use Otsu so you don't choose manually)
# ------------------------------------
_, th = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# ------------------------------------
# Morphological refinement
# ------------------------------------
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
th_close = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel)  # fill holes
th_open = cv2.morphologyEx(th_close, cv2.MORPH_OPEN, kernel)  # remove noise

# ------------------------------------
# Extract melt pool coordinates
# ------------------------------------
coords = np.column_stack(np.where(th_open > 0))

# Get extreme points
topmost    = coords[np.argmin(coords[:, 0])]
bottommost = coords[np.argmax(coords[:, 0])]
leftmost   = coords[np.argmin(coords[:, 1])]
rightmost  = coords[np.argmax(coords[:, 1])]

# Compute width & height
width  = rightmost[1] - leftmost[1]
height = bottommost[0] - topmost[0]

# Compute centroid
centroid_x = int(np.mean(coords[:, 1]))
centroid_y = int(np.mean(coords[:, 0]))

# ------------------------------------
# Draw results on a color version
# ------------------------------------
img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

# Draw bounding box
cv2.rectangle(img_color,
              (leftmost[1], topmost[0]),
              (rightmost[1], bottommost[0]),
              (0, 0, 255), 2)

# Draw centroid
cv2.circle(img_color, (centroid_x, centroid_y), 5, (0, 255, 0), -1)

# ------------------------------------
# Print measurements
# ------------------------------------
print("Melt pool width:", width)
print("Melt pool height:", height)
print("Centroid:", (centroid_x, centroid_y))

# ------------------------------------
# Show results
# ------------------------------------
cv2.imshow("Original", img)
cv2.imshow("Binary segmentation", th_open)
cv2.imshow("Measured melt pool", img_color)

cv2.waitKey(0)
cv2.destroyAllWindows()